In [16]:
import cv2
import numpy as np
import mediapipe as mp
import math

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
cap.set(3, 840)
cap.set(4, 720)

pt1 = (100, 100)
pt2 = (150, 150)
pt3 = (250, 250)
pt4 = (150, 100)
col_rec = (255, 0, 0, 100)
col_rec1 = (255, 0, 0, 100)

class Rectangle:
    def __init__(self, pos, size=[200, 200], color=(255, 0, 0, 100)):
        self.pos = pos
        self.size = size
        self.color = color
    
    def update(self, cursor):
        pt1 = self.pos
        pt2 = self.size
        
        if pt1[0] < cursor_x < pt1[0] + pt2[0] and pt1[1] < cursor_y < pt1[1] + pt2[1]:
            col_rec = (0, 255, 0, 100) 
            dx = cursor_x - pt1[0] - pt2[0] // 2
            dy = cursor_y - pt1[1] - pt2[1] // 2
            pt1 = (pt1[0] + dx, pt1[1] + dy)
            pt2 = (pt2[0], pt2[1])
            col_rec = (110, 255, 0, 100)
        else:
            col_rec = (255, 0, 0, 100)
        
        return pt1, pt2, col_rec

with mp_hands.Hands(
    min_detection_confidence=0.8,
    min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        success, image = cap.read()

        if not success:
            print("Ignoring empty camera frame.")
            continue

        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                if len(hand_landmarks.landmark) >= 9:
                    middle_tip = (hand_landmarks.landmark[12].x, hand_landmarks.landmark[12].y,
                                 hand_landmarks.landmark[12].z)
                    index_tip = (hand_landmarks.landmark[8].x, hand_landmarks.landmark[8].y,
                                 hand_landmarks.landmark[8].z)
                    distance = math.sqrt(
                        (middle_tip[0] - index_tip[0]) ** 2 + (middle_tip[1] - index_tip[1]) ** 2 + (
                                    middle_tip[2] - index_tip[2]) ** 2)
                    
                    cursor = hand_landmarks.landmark[8]
                    cursor_x = int(cursor.x * image.shape[1])
                    cursor_y = int(cursor.y * image.shape[0])
                    
                    rect = Rectangle(pt1, pt2, col_rec)
                    rect1 = Rectangle(pt3, pt4, col_rec1)
                    
                    if distance * 1000 < 40:
                        pt1, pt2, col_rec = rect.update(cursor)
                        pt3, pt4, col_rec1 = rect1.update(cursor)
                        
        overlay = image.copy()
        
        # Draw the first rectangle
        cv2.rectangle(overlay, pt1, (pt1[0] + pt2[0], pt1[1] + pt2[1]), col_rec, cv2.FILLED)
        cv2.rectangle(overlay, pt1, (pt1[0] + pt2[0], pt1[1] + pt2[1]), (0, 0, 255), 2)  # Add border
        
        # Draw the second rectangle
        cv2.rectangle(overlay, pt3, (pt3[0] + pt4[0], pt3[1] + pt4[1]), col_rec1, cv2.FILLED)
        cv2.rectangle(overlay, pt3, (pt3[0] + pt4[0], pt3[1] + pt4[1]), (0, 0, 255), 2)  # Add border
        
        alpha = 0.5  # Opacity value (0.0 - fully transparent, 1.0 - fully opaque)
        image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)

        cv2.imshow('Hand Movement', image)
        
        if cv2.waitKey(5) & 0xFF == 27:
            cv2.destroyAllWindows()
            break

cap.release()

